<a href="https://colab.research.google.com/github/mateusduarte-max/Projetos_Python/blob/main/Envio_pesquisas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importe bibliotecas
import pandas as pd
import datetime

# Bibliotecas Google Golab
from google.colab import auth
auth.authenticate_user()
# Bibliotecas Google Sheet
import gspread
from oauth2client.client import GoogleCredentials
from gspread_dataframe import set_with_dataframe

from google.colab import drive
drive.mount('/content/drive')

# Autenticação Final
gc = gspread.authorize(GoogleCredentials.get_application_default())

Mounted at /content/drive


In [ ]:
# Importe base
baseEnviopesquisas = pd.read_csv('/content/BuscaAtendimentos_20210630_085423.csv', sep=';', encoding='ISO-8859-1', usecols=['Fila_Atual','Protocolo','email', 'Operador', 'Data Fim Atendimento', 'Classificação','Canal', 'Detalhe Estado']).fillna('')
# Excluir carater protocolo
baseEnviopesquisas['Protocolo'] = baseEnviopesquisas["Protocolo"].str.replace('-', '')
# Criar Coluna De Envio
baseEnviopesquisas['data_do_envio'] = datetime.datetime.today().strftime('%Y-%m-%d')
# Criar coluna Protoloco + Data fim atendimento
baseEnviopesquisas['Data_Fim Atendimento_1'] = baseEnviopesquisas['Data Fim Atendimento'].str.replace('/','')
baseEnviopesquisas['id_transacao'] = baseEnviopesquisas['Protocolo'] + baseEnviopesquisas['Data_Fim Atendimento_1']
# Criar coluna de telefone
baseEnviopesquisas['telefone'] = ' '
# Criar coluna de tipo solicitado
baseEnviopesquisas['tipo_solicitacao'] = ' '
# Criar coluna de subclassificacao
baseEnviopesquisas['subclassificacao_base'] = ' '
# Criar coluna de data_criacao_ticket
baseEnviopesquisas['data_criacao_ticket'] = ' '
# Criar coluna de id_do_pedido_txt
baseEnviopesquisas['id_do_pedido_txt'] = ' '
# Criar coluna de id_do_seller
baseEnviopesquisas['id_do_seller'] = ' '
# Criar coluna de id cliente
baseEnviopesquisas['id_cliente'] = baseEnviopesquisas['email']
# Criar coluna de nome
baseEnviopesquisas['nome'] = baseEnviopesquisas['email']
# Criar coluna de nome
baseEnviopesquisas['id_consultor'] = baseEnviopesquisas['Operador'].str.upper()
# Criar coluna de nome
baseEnviopesquisas['data_transacao'] = baseEnviopesquisas['Data Fim Atendimento']
# Criar coluna Jornada
baseEnviopesquisas['jornada'] = 'CSAT NETS'


#Tratativa Primeiro Nome
nomes = baseEnviopesquisas['Operador']
n = [nome.title().strip().split(' ')[0] for nome in nomes]
baseEnviopesquisas['Primeiro_Nome'] = pd.DataFrame(data=n)

# Renomear coluna
baseEnviopesquisas.rename(columns={'Fila_Atual': 'motivo', 'Protocolo':'protocolo', 'Classificação':'classificacao_base', 'Canal':'canal_de_entrada','Detalhe Estado':'Detalhe_Estado'}, inplace = True)

# Filtro pesquisas respondidas

baseEnviopesquisasFiltro = baseEnviopesquisas.query('Detalhe_Estado=="Respondido pelo operador"')

#Selecionar colunas ordem
baseEnviopesquisasNovo = baseEnviopesquisasFiltro[['data_do_envio', 
                                                 'motivo', 
                                                 'protocolo', 
                                                 'id_transacao', 
                                                 'email', 
                                                 'id_cliente', 
                                                 'telefone', 
                                                 'nome', 
                                                 'id_consultor', 
                                                 'Primeiro_Nome', 
                                                 'data_transacao', 
                                                 'tipo_solicitacao',
                                                 'classificacao_base',
                                                 'jornada',
                                                 'subclassificacao_base',
                                                 'canal_de_entrada',
                                                 'data_criacao_ticket',
                                                 'id_do_pedido_txt',
                                                 'id_do_seller']]


baseEnviopesquisasNovo.tail()

,data_do_envio,motivo,protocolo,id_transacao,email,id_cliente,telefone,nome,id_consultor,Primeiro_Nome,data_transacao,tipo_solicitacao,classificacao_base,jornada,subclassificacao_base,canal_de_entrada,data_criacao_ticket,id_do_pedido_txt,id_do_seller
1798,2021-06-30,Casos Críticos,9862169412,986216941229062021,carmocanongia@gmail.com,carmocanongia@gmail.com,,carmocanongia@gmail.com,GABRIEL LOPES CARDOSO,Gabriel,29/06/2021,,Fidelização,CSAT NETS,,DTMail,,,
1799,2021-06-30,Casos Críticos,9862203972,986220397229062021,isabelapsi.lobo@gmail.com,isabelapsi.lobo@gmail.com,,isabelapsi.lobo@gmail.com,GABRIEL LOPES CARDOSO,Gabriel,29/06/2021,,Fidelização,CSAT NETS,,DTMail,,,
1800,2021-06-30,Casos Críticos,9862209222,986220922229062021,paolacnicolau@gmail.com,paolacnicolau@gmail.com,,paolacnicolau@gmail.com,GABRIEL LOPES CARDOSO,Gabriel,29/06/2021,,Fidelização,CSAT NETS,,DTMail,,,
1801,2021-06-30,Casos Críticos,9862219342,986221934229062021,valerialeticiaalves@gmail.com,valerialeticiaalves@gmail.com,,valerialeticiaalves@gmail.com,GABRIEL LOPES CARDOSO,Gabriel,29/06/2021,,Fidelização,CSAT NETS,,DTMail,,,
1802,2021-06-30,Casos Críticos,9862222922,986222292229062021,pedro.silvaoliveira1005@gmail.com,pedro.silvaoliveira1005@gmail.com,,pedro.silvaoliveira1005@gmail.com,GABRIEL LOPES CARDOSO,Gabriel,29/06/2021,,Fidelização,CSAT NETS,,DTMail,,,


In [ ]:
# abrir arquivo google
arquivo = gc.open('Consulta_quadro')
# abrir planilha 'aba'
pag_extracao = arquivo.worksheet('Extração 1')
# colocar a aba dentro de um data_frame
base_quadro_bq = pd.DataFrame(pag_extracao.get_all_records())

filtro = base_quadro_bq['id'] != 'ID'
base_quadro_bq_novo = base_quadro_bq[filtro]
base_quadro_bq_novo['ID'] = base_quadro_bq_novo['id'].astype(int)
base_quadro_bq_novo['id_consultor'] = base_quadro_bq_novo['nome']

#Visualização
base_quadro_bq_novo.head()

,area_macro,id,nome,departamento,turno,alocacao,status,extra,horario_de_trabalho,carga_horaria,forma_de_mensurar,meta,id_supervisor,supervisor,id_coordenador,coordenador,observacao,login,dt_updated,ID,id_consultor
0,NPS,255953,LAIS OLIVEIRA DE CASTRO,NPS MAGALU,MATUTINO,HOME OFFICE,FÉRIAS,-,-,,Sem Mensuração,0,134527,THANUSE MAIANE SOUZA SILVA,307683,RAPHAEL KUNIO IWANAGA SOARES,,Verificar ID,07/06/2021 10:50:59,255953,LAIS OLIVEIRA DE CASTRO
1,NPS,278709,EDUARDA MANHANI AMBROSIO,NPS NETS,MATUTINO,HOME OFFICE,FÉRIAS,-,-,,Sem Mensuração,0,134527,THANUSE MAIANE SOUZA SILVA,307683,RAPHAEL KUNIO IWANAGA SOARES,,Verificar ID,07/06/2021 10:50:59,278709,EDUARDA MANHANI AMBROSIO
2,Ativo_Suporte,141319,JOSELAINE DE MATOS VALENTE PER,Suporte/ Ativo,MATUTINO,HOME OFFICE,Afastado,-,-,,Sem Mensuração,0,186023,LAWENNY KEROLA DE OLIVEIRA,92412,JUSCELENE DE SOUSA NASCIMENTO,,Verificar ID,07/06/2021 10:50:59,141319,JOSELAINE DE MATOS VALENTE PER
3,Ativo_Suporte,149777,LAISA CAMARGO MARTINS,Suporte/ Ativo,MATUTINO,HOME OFFICE,Afastado,-,-,,Sem Mensuração,0,186023,LAWENNY KEROLA DE OLIVEIRA,92412,JUSCELENE DE SOUSA NASCIMENTO,,Verificar ID,07/06/2021 10:50:59,149777,LAISA CAMARGO MARTINS
4,Voz_Nets,284387,GABRIEL HENRIQUE DA SILVA,Lgpd Nets,MATUTINO,HOME OFFICE,Ativo,-,07:40:00,06:20,Performance: Soma da Quantidade de Contatos di...,45,146346,THALES GUSTAVO SALGADO DA SILV,138482,ALINE CRISTINA DA SILVA,,ARIN_SOUSA,07/06/2021 10:50:59,284387,GABRIEL HENRIQUE DA SILVA


In [ ]:
# junção bases
conferencia = pd.merge(baseEnviopesquisasNovo, base_quadro_bq_novo, how='left', on='id_consultor').fillna('')
conferencia['id_loja'] = conferencia['supervisor']
conferencia['canal'] = conferencia['departamento']
conferencia['id_agente_magalu'] = conferencia['id']
conferencia.tail()

,data_do_envio,motivo,protocolo,id_transacao,email,id_cliente,telefone,nome_x,id_consultor,Primeiro_Nome,data_transacao,tipo_solicitacao,classificacao_base,jornada,subclassificacao_base,canal_de_entrada,data_criacao_ticket,id_do_pedido_txt,id_do_seller,area_macro,id,nome_y,departamento,turno,alocacao,status,extra,horario_de_trabalho,carga_horaria,forma_de_mensurar,meta,id_supervisor,supervisor,id_coordenador,coordenador,observacao,login,dt_updated,ID,id_loja,canal,id_agente_magalu
1691,2021-06-30,Casos Críticos,9862169412,986216941229062021,carmocanongia@gmail.com,carmocanongia@gmail.com,,carmocanongia@gmail.com,GABRIEL LOPES CARDOSO,Gabriel,29/06/2021,,Fidelização,CSAT NETS,,DTMail,,,,DIGITAL,276032,GABRIEL LOPES CARDOSO,E-mail Netshoes,MATUTINO,HOME OFFICE,Ativo,-,07:40:00,07:32,Soma da quantidade de e-mails tratados no dia,120,212075,JOAO PEDRO DA SILVA FERREIRA,275350,ANDREIA DE FATIMA GARCIA DOS S,,RHA_MOREIRA,07/06/2021 10:50:59,276032,JOAO PEDRO DA SILVA FERREIRA,E-mail Netshoes,276032
1692,2021-06-30,Casos Críticos,9862203972,986220397229062021,isabelapsi.lobo@gmail.com,isabelapsi.lobo@gmail.com,,isabelapsi.lobo@gmail.com,GABRIEL LOPES CARDOSO,Gabriel,29/06/2021,,Fidelização,CSAT NETS,,DTMail,,,,DIGITAL,276032,GABRIEL LOPES CARDOSO,E-mail Netshoes,MATUTINO,HOME OFFICE,Ativo,-,07:40:00,07:32,Soma da quantidade de e-mails tratados no dia,120,212075,JOAO PEDRO DA SILVA FERREIRA,275350,ANDREIA DE FATIMA GARCIA DOS S,,RHA_MOREIRA,07/06/2021 10:50:59,276032,JOAO PEDRO DA SILVA FERREIRA,E-mail Netshoes,276032
1693,2021-06-30,Casos Críticos,9862209222,986220922229062021,paolacnicolau@gmail.com,paolacnicolau@gmail.com,,paolacnicolau@gmail.com,GABRIEL LOPES CARDOSO,Gabriel,29/06/2021,,Fidelização,CSAT NETS,,DTMail,,,,DIGITAL,276032,GABRIEL LOPES CARDOSO,E-mail Netshoes,MATUTINO,HOME OFFICE,Ativo,-,07:40:00,07:32,Soma da quantidade de e-mails tratados no dia,120,212075,JOAO PEDRO DA SILVA FERREIRA,275350,ANDREIA DE FATIMA GARCIA DOS S,,RHA_MOREIRA,07/06/2021 10:50:59,276032,JOAO PEDRO DA SILVA FERREIRA,E-mail Netshoes,276032
1694,2021-06-30,Casos Críticos,9862219342,986221934229062021,valerialeticiaalves@gmail.com,valerialeticiaalves@gmail.com,,valerialeticiaalves@gmail.com,GABRIEL LOPES CARDOSO,Gabriel,29/06/2021,,Fidelização,CSAT NETS,,DTMail,,,,DIGITAL,276032,GABRIEL LOPES CARDOSO,E-mail Netshoes,MATUTINO,HOME OFFICE,Ativo,-,07:40:00,07:32,Soma da quantidade de e-mails tratados no dia,120,212075,JOAO PEDRO DA SILVA FERREIRA,275350,ANDREIA DE FATIMA GARCIA DOS S,,RHA_MOREIRA,07/06/2021 10:50:59,276032,JOAO PEDRO DA SILVA FERREIRA,E-mail Netshoes,276032
1695,2021-06-30,Casos Críticos,9862222922,986222292229062021,pedro.silvaoliveira1005@gmail.com,pedro.silvaoliveira1005@gmail.com,,pedro.silvaoliveira1005@gmail.com,GABRIEL LOPES CARDOSO,Gabriel,29/06/2021,,Fidelização,CSAT NETS,,DTMail,,,,DIGITAL,276032,GABRIEL LOPES CARDOSO,E-mail Netshoes,MATUTINO,HOME OFFICE,Ativo,-,07:40:00,07:32,Soma da quantidade de e-mails tratados no dia,120,212075,JOAO PEDRO DA SILVA FERREIRA,275350,ANDREIA DE FATIMA GARCIA DOS S,,RHA_MOREIRA,07/06/2021 10:50:59,276032,JOAO PEDRO DA SILVA FERREIRA,E-mail Netshoes,276032


In [ ]:
# Remover Coporativo e email Nets
conferencia['Validacao_motivo'] = conferencia['motivo'].str.contains('Corporativo')
conferencia_1 = conferencia.query("Validacao_motivo==False")
conferencia_1['Validacao_email'] = conferencia['email'].str.contains('@netshoes')
conferencia_2 = conferencia_1.query("Validacao_email==False")
conferencia_2.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,data_do_envio,motivo,protocolo,id_transacao,email,id_cliente,telefone,nome_x,id_consultor,Primeiro_Nome,data_transacao,tipo_solicitacao,classificacao_base,jornada,subclassificacao_base,canal_de_entrada,data_criacao_ticket,id_do_pedido_txt,id_do_seller,area_macro,id,nome_y,departamento,turno,alocacao,status,extra,horario_de_trabalho,carga_horaria,forma_de_mensurar,meta,id_supervisor,supervisor,id_coordenador,coordenador,observacao,login,dt_updated,ID,id_loja,canal,id_agente_magalu,Validacao_motivo,Validacao_email
0,2021-06-30,Fila Padrão,9859215842,985921584229062021,neris704@gmail.com,neris704@gmail.com,,neris704@gmail.com,RAFAELLA FERREIRA GOMES FIORI SILVA,Rafaella,29/06/2021,,Fidelização,CSAT NETS,,DTMail,,,,,,,,,,,,,,,,,,,,,,,,,,,False,False
1,2021-06-30,Fila Padrão,9859206082,985920608229062021,mauricioliberato131@gmail.com,mauricioliberato131@gmail.com,,mauricioliberato131@gmail.com,SANDRA HELENA ALVES ARANHA RAMOS,Sandra,29/06/2021,,Fidelização,CSAT NETS,,DTMail,,,,,,,,,,,,,,,,,,,,,,,,,,,False,False
2,2021-06-30,Fila Padrão,9859306312,985930631229062021,cjdeljardel@gmail.com,cjdeljardel@gmail.com,,cjdeljardel@gmail.com,PATRICK COSTA PATRINIANI,Patrick,29/06/2021,,Fidelização,CSAT NETS,,DTMail,,,,Bko3P,275439,PATRICK COSTA PATRINIANI,Backoffice 3p Nets,MATUTINO,Home Office,ATIVO,-,08:00| 13:00- 13:20| 14:25,06:20,Aderência a Escala ( Média do % de aderência a...,"0,99",274440,SUSANA APARECIDA VILELA FIGUER,274385,Mariana Silva Cedran,,Verificar ID,07/06/2021 10:50:59,275439,SUSANA APARECIDA VILELA FIGUER,Backoffice 3p Nets,275439,False,False
3,2021-06-30,Fila Padrão,9859208602,985920860229062021,suellensilvaleite6@gmail.com,suellensilvaleite6@gmail.com,,suellensilvaleite6@gmail.com,SANDRA HELENA ALVES ARANHA RAMOS,Sandra,29/06/2021,,Fidelização,CSAT NETS,,DTMail,,,,,,,,,,,,,,,,,,,,,,,,,,,False,False
4,2021-06-30,Fila Padrão,9859274212,985927421229062021,oliwessil@mail.uft.edu.br,oliwessil@mail.uft.edu.br,,oliwessil@mail.uft.edu.br,SANDRA FERREIRA MARCIANO,Sandra,29/06/2021,,Fidelização,CSAT NETS,,DTMail,,,,DIGITAL,274425,SANDRA FERREIRA MARCIANO,E-mail Netshoes,MATUTINO,HOME OFFICE,Ativo,-,07:40:00,06:20,Soma da quantidade de e-mails tratados no dia,100,212075,JOAO PEDRO DA SILVA FERREIRA,275350,ANDREIA DE FATIMA GARCIA DOS S,,TAIB_SOUZA,07/06/2021 10:50:59,274425,JOAO PEDRO DA SILVA FERREIRA,E-mail Netshoes,274425,False,False


In [ ]:
#Selecionar colunas ordem
conferenciaFinal = conferencia_2[['data_do_envio', 
                                                 'motivo', 
                                                 'protocolo', 
                                                 'id_transacao', 
                                                 'email', 
                                                 'id_cliente', 
                                                 'telefone', 
                                                 'nome_x', 
                                                 'id_consultor', 
                                                 'Primeiro_Nome',
                                                 'id_loja',
                                                 'canal', 
                                                 'data_transacao', 
                                                 'tipo_solicitacao',
                                                 'classificacao_base',
                                                 'jornada',
                                                 'subclassificacao_base',
                                                 'coordenador',
                                                 'id_agente_magalu',
                                                 'canal_de_entrada',
                                                 'data_criacao_ticket',
                                                 'id_do_pedido_txt',
                                                 'id_do_seller']]

# Renomear coluna
conferenciaFinal.rename(columns={'nome_x': 'nome'}, inplace = True) 

# transformar coluna
conferenciaFinal['id_agente_magalu'].astype(str)

envio = conferenciaFinal.drop_duplicates('id_transacao', keep=False)

envio.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,data_do_envio,motivo,protocolo,id_transacao,email,id_cliente,telefone,nome,id_consultor,Primeiro_Nome,id_loja,canal,data_transacao,tipo_solicitacao,classificacao_base,jornada,subclassificacao_base,coordenador,id_agente_magalu,canal_de_entrada,data_criacao_ticket,id_do_pedido_txt,id_do_seller
0,2021-06-30,Fila Padrão,9859215842,985921584229062021,neris704@gmail.com,neris704@gmail.com,,neris704@gmail.com,RAFAELLA FERREIRA GOMES FIORI SILVA,Rafaella,,,29/06/2021,,Fidelização,CSAT NETS,,,,DTMail,,,
1,2021-06-30,Fila Padrão,9859206082,985920608229062021,mauricioliberato131@gmail.com,mauricioliberato131@gmail.com,,mauricioliberato131@gmail.com,SANDRA HELENA ALVES ARANHA RAMOS,Sandra,,,29/06/2021,,Fidelização,CSAT NETS,,,,DTMail,,,
2,2021-06-30,Fila Padrão,9859306312,985930631229062021,cjdeljardel@gmail.com,cjdeljardel@gmail.com,,cjdeljardel@gmail.com,PATRICK COSTA PATRINIANI,Patrick,SUSANA APARECIDA VILELA FIGUER,Backoffice 3p Nets,29/06/2021,,Fidelização,CSAT NETS,,Mariana Silva Cedran,275439,DTMail,,,
3,2021-06-30,Fila Padrão,9859208602,985920860229062021,suellensilvaleite6@gmail.com,suellensilvaleite6@gmail.com,,suellensilvaleite6@gmail.com,SANDRA HELENA ALVES ARANHA RAMOS,Sandra,,,29/06/2021,,Fidelização,CSAT NETS,,,,DTMail,,,
4,2021-06-30,Fila Padrão,9859274212,985927421229062021,oliwessil@mail.uft.edu.br,oliwessil@mail.uft.edu.br,,oliwessil@mail.uft.edu.br,SANDRA FERREIRA MARCIANO,Sandra,JOAO PEDRO DA SILVA FERREIRA,E-mail Netshoes,29/06/2021,,Fidelização,CSAT NETS,,ANDREIA DE FATIMA GARCIA DOS S,274425,DTMail,,,


In [ ]:
# Salvar base geral no drive
def input_drive_nets():
  #Abrir planilha e primeira aba
  pagina = gc.open('Envio_Pesquisas').sheet1
  # Limpar base
  pagina.clear()
  # Importar dataframe na planilha
  return set_with_dataframe(pagina, envio)


input_drive_nets()

In [ ]:
conferenciaFinal.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1692 entries, 0 to 1695
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   data_do_envio          1692 non-null   object
 1   motivo                 1692 non-null   object
 2   protocolo              1692 non-null   object
 3   id_transacao           1692 non-null   object
 4   email                  1692 non-null   object
 5   id_cliente             1692 non-null   object
 6   telefone               1692 non-null   object
 7   nome                   1692 non-null   object
 8   id_consultor           1692 non-null   object
 9   Primeiro_Nome          1692 non-null   object
 10  id_loja                1692 non-null   object
 11  canal                  1692 non-null   object
 12  data_transacao         1692 non-null   object
 13  tipo_solicitacao       1692 non-null   object
 14  classificacao_base     1692 non-null   object
 15  jornada              